# Arbitrage auto detection using matrices
 
 https://papers.ssrn.com/sol3/papers.cfm?abstract_id=1096549
 
 [original paper method](https://d1wqtxts1xzle7.cloudfront.net/34802958/06F167EF-B243-48ED-8C45-F7466B3136EB-WebPublishings-How_to_make_decision_AHP.pdf?1411208782=&response-content-disposition=inline%3B+filename%3DHow_to_make_a_decision_The_Analytic_Hier.pdf&Expires=1616709686&Signature=TPWpLboiXquvLmKWLCmdRi74uUtv7J4vrR0qXa4uFRjQpoEuZ0YBYZdnmg8xtTy8IJJjkaTV6MK52iNaUwEPn~qi67Sw1J2zx38o8zlAkwmNqJ2MZA5vn75Tk3NEm2jCpcCrjyn~UgUYXyZsKdf8007fAxrLIRZ~vS9Lj91tvZh2QUYpzp9CKqu9osR9lkCWu0SreKRYEnu~XTS0gvocjbt1gzYtQ7wpFG-QDih-7gNszyreN1G5IJBUg4x5L8dUBSXEA48UYoWn84gLoOcnDpY7PYPpuvTSNCr~HdymMH3MRJvuJtpoE10Qx058pdjHzPd2hhqLW8JxE8TBLKLC-w__&Key-Pair-Id=APKAJLOHF5GGSLRBV4ZA)

In [3]:
import asyncio
import aiohttp
from aiohttp import ClientSession

import urllib
import urllib.request

import time
import json

import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler

In [4]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [5]:
def compute_API(lambda_max, n):
    return np.abs(lambda_max-n)/(n-1)

def clock_ms():
    return time.perf_counter()*1000

In [6]:
def get_top_symbols_market_cap(n):
    # market cap
    binance_uri = "https://www.binance.com/exchange-api/v2/public/asset-service/product/get-products"
    web_url = urllib.request.urlopen(binance_uri)
    data = web_url.read()
    encoding = web_url.info().get_content_charset('utf-8')
    JSON_object = json.loads(data.decode(encoding))
    df_market = pd.DataFrame(JSON_object['data'])
    df_market['market_cap'] = df_market['cs'].fillna(0).astype(float)*df_market['c'].fillna(0).astype(float)
    top_n = df_market.sort_values(by='market_cap', ascending=False)[:n]
    assets = set(top_n['b'].values).union(set(top_n['q'].values))
    symbols = top_n['s'].to_list()
    return symbols, assets

In [7]:
def calculate_arbitrage(A, vecmax, fees, currencies):
    locations = currencies
    B = np.fromfunction(lambda i, j: vecmax[i] / vecmax[j], (len(A),len(A)), dtype=int)
    C = np.divide(A, B)

    C_max = np.unravel_index(C.argmax(), C.shape)
    C_min = np.unravel_index(C.argmin(), C.shape)

    # Algorithm
    # BUY = division
    # SELL = multiply
    print('Arbitrage orders: ', end="")
    if C_max[0]==C_min[1] and C_max[1]==C_min[0]:
        # Direct arbitrage
        # Use currency C_min[1] and buy currency C_min[0] in location C_min[1]
        # and sell it for currency C_max[0] in location C_max[1]
        print('DIRECT')
        aer = 1/np.real(C[C_min]*C[C_max])-1

        print(f'{currencies[C_min[1]]} -> {currencies[C_min[0]]} -> {currencies[C_min[1]]} : AER = {aer:.3%} : Fee = {fees*2:.3%} : Return = {aer-fees*2:.3%}')
        print(f'BUY  {currencies[C_min[0]]}/{currencies[C_min[1]]}({A[C_min]}) in {locations[C_min[1]]:6}')
        print(f'SELL {currencies[C_max[0]]}/{currencies[C_min[0]]}({A[C_max[0],C_min[0]]}) in {locations[C_max[1]]:6}')

        operation = 1/(A[C_min]*A[C_max])
        print(f'1/{A[C_min]:.4}*{A[C_max]:.4} = {operation:.5}')

    elif C_max[0]==C_min[0] or C_max[1]==C_min[1]:
        # Triangular arbitrage
        if C_max[0]==C_min[0]:
            # Arbitrage elements in the same row
            # Use currency C_min[1] and buy currency C_min[0] in location C_min[1]
            # then sell it for currency C_max[1] in location C_max[1]
            # then buy currency C_max[1] in location C_max[1]
            print('TRIANGULAR ROW')
            aer = np.real(1/C[C_min]*C[C_min[0],C_max[1]]/C[C_min[1],C_max[1]]-1)

            print(f'{currencies[C_min[1]]} -> {currencies[C_min[0]]} -> {currencies[C_max[1]]} -> {currencies[C_min[1]]} : AER = {aer:.3%} : Fee = {fees*3:.3%} : Return = {aer-fees*3:.3%}')
            print(f'BUY  {currencies[C_min[0]]}/{currencies[C_min[1]]}({A[C_min]}) in {locations[C_min[1]]}')
            print(f'SELL {currencies[C_min[0]]}/{currencies[C_max[1]]}({A[C_min[0],C_max[1]]}) in {locations[C_max[1]]}')
            print(f'BUY  {currencies[C_min[1]]}/{currencies[C_max[1]]}({A[C_min[1],C_max[1]]}) in {locations[C_max[1]]}')
            operation = 1/A[C_min]*A[C_min[0],C_max[1]]/A[C_min[1],C_max[1]]
            print(f'1/{A[C_min]:.4}*{A[C_min[0],C_max[1]]:.4}/{A[C_min[1],C_max[1]]:.4} = {operation:.5}')
        else: # C_max[1]==C_min[1]
            # Arbitrage elements in the same col
            # Use currency C_min[1] and buy currency C_min[0] in location C_min[1]
            # then sell it for currency C_max[0] in location C_max[0]
            # then sell it for currency C_min[1] in location C_max[1]
            print('TRIANGULAR COLUMN')
            aer = np.real(1/C[C_min]*C[C_min[0], C_max[0]]*C[C_max[0],C_max[1]]-1)

            print(f'{currencies[C_min[1]]} -> {currencies[C_min[0]]} -> {currencies[C_max[0]]} -> {currencies[C_min[1]]} : AER = {aer:.3%} : Fee = {fees*3:.3%} : Return = {aer-fees*3:.3%}')
            print(f'BUY {currencies[C_min[0]]}/{currencies[C_min[1]]}({A[C_min]}) in {locations[C_min[1]]}')
            print(f'SELL {currencies[C_min[0]]}/{currencies[C_max[0]]}({A[C_min[0],C_max[0]]}) in {locations[C_max[0]]}')
            print(f'SELL {currencies[C_max[0]]}/{currencies[C_min[1]]}({A[C_max[0],C_min[1]]}) in {locations[C_max[1]]}')
            operation = 1/A[C_min]*A[C_min[0],C_max[0]]*A[C_max[0],C_max[1]]
            print(f'1/{A[C_min]:.4}*{A[C_min[0],C_max[0]]:.4}*{A[C_max[0],C_max[1]]:.4} = {operation:.5}')
    else:
        # Cuadrangular arbitrage
        # Arbitrage that involves four currencies and four locations
        # Use currency C_min[1] and buy currency C_min[0] in location C_min[1]
        # then sell it for currency C_max[0] in location C_max[0]
        # then sell it for currency C_max[1] in location C_max[1]
        # then buy currency C_min[1] in location C_max[1]
        print('CUADRANGULAR')
        aer = np.real(1/C[C_min]*C[C_min[0],C_max[0]]*C[C_max]/C[C_min[1],C_max[1]]-1)

        print(f'{currencies[C_min[1]]} -> {currencies[C_min[0]]} -> {currencies[C_max[0]]} -> {currencies[C_max[1]]} -> {currencies[C_min[1]]} : AER = {aer:.3%} : Fee = {fees*4:.3%} : Return = {aer-fees*4:.3%}')
        print(f'BUY  {currencies[C_min[0]]}/{currencies[C_min[1]]}({A[C_min]}) in {locations[C_min[1]]}')
        print(f'SELL {currencies[C_min[0]]}/{currencies[C_max[0]]}({A[C_min[0],C_max[0]]}) in {locations[C_max[0]]}')
        print(f'SELL {currencies[C_max[0]]}/{currencies[C_max[1]]}({A[C_max[0],C_max[1]]}) in {locations[C_max[1]]}')
        print(f'BUY  {currencies[C_min[1]]}/{currencies[C_max[1]]}({A[C_min[1],C_max[1]]}) in {locations[C_max[1]]}')
        operation = 1/A[C_min]*A[C_min[0],C_max[0]]*A[C_max[0],C_max[1]]/A[C_min[1],C_max[1]]
        print(f'1/{A[C_min]:.4}*{A[C_min[0],C_max[0]]:.4}*{A[C_max[0],C_max[1]]:.4}/{A[C_min[1],C_max[1]]:.4} = {operation:.5}')
        
    return aer

In [8]:
binance_base = "https://<>.binance.com"
binance_subdomains = ["api", "api1", "api2", "api3"]

binance_url = binance_base.replace('<>', binance_subdomains[0])

binance_endpoints = {
    'ping': ('GET', '/api/v3/ping'),
    'server_time': ('GET', '/api/v3/time'),
    'exchange_info': ('GET', '/api/v3/exchangeInfo'),
    'order_book': ('GET', '/api/v3/depth', {'symbol': True, 'limit': False}),
    'recent_trades': ('GET', '/api/v3/trades', {'symbol': True, 'limit': False}),
    'average_price': ('GET', '/api/v3/avgPrice', {'symbol': True}),
    'price': ('GET', '/api/v3/ticker/price', {'symbol': False}),
    'best_book_price': ('GET', '/api/v3/ticker/bookTicker', {'symbol': False})
}

In [9]:
binance_uri = binance_url + binance_endpoints['exchange_info'][1]

web_url = urllib.request.urlopen(binance_uri)
data = web_url.read()
encoding = web_url.info().get_content_charset('utf-8')
JSON_object = json.loads(data.decode(encoding))

df_symbols = pd.DataFrame(JSON_object['symbols'])
df_symbols = df_symbols[df_symbols['status']=='TRADING']

In [10]:
dict(web_url.info())

{'Content-Type': 'application/json;charset=UTF-8',
 'Content-Length': '1514028',
 'Connection': 'close',
 'Date': 'Sat, 03 Apr 2021 14:33:12 GMT',
 'Server': 'nginx',
 'Vary': 'Accept-Encoding',
 'x-mbx-uuid': '20ce1526-8f90-40f4-9f94-c2c3d15c7767',
 'x-mbx-used-weight': '1',
 'x-mbx-used-weight-1m': '1',
 'Strict-Transport-Security': 'max-age=31536000; includeSubdomains',
 'X-Frame-Options': 'SAMEORIGIN',
 'X-Xss-Protection': '1; mode=block',
 'X-Content-Type-Options': 'nosniff',
 'Content-Security-Policy': "default-src 'self'",
 'X-Content-Security-Policy': "default-src 'self'",
 'X-WebKit-CSP': "default-src 'self'",
 'Cache-Control': 'no-cache, no-store, must-revalidate',
 'Pragma': 'no-cache',
 'Expires': '0',
 'Access-Control-Allow-Origin': '*',
 'Access-Control-Allow-Methods': 'GET, HEAD, OPTIONS',
 'X-Cache': 'Miss from cloudfront',
 'Via': '1.1 322aee8aa5d1b8e91a9db30a8244f179.cloudfront.net (CloudFront)',
 'X-Amz-Cf-Pop': 'CDG3-C1',
 'X-Amz-Cf-Id': '9YIPu070hPu3A6hDhrLTBbQxjeA

In [41]:
assets = sorted(list(set(df_symbols['baseAsset'].unique()).union(set(df_symbols['quoteAsset'].unique()))))
symbols = sorted(list(df_symbols['symbol'].unique()))

#symbols, assets = get_top_symbols_market_cap(100)

forbiden_symbols = ['REN']
for fs in forbiden_symbols:
    assets = list(filter(lambda x: fs not in x, assets))
    symbols = list(filter(lambda x: fs not in x, symbols))

len(assets), len(symbols)

(344, 1011)

---

---

### try arbitrage algorithm

---

# Try recurrent arbitrage

In [32]:
async def update_symbol_prices(q, symbol):
    binance_uri = binance_url + binance_endpoints['order_book'][1] + f'?symbol={symbol}&limit=5'

    while True:
        async with ClientSession() as session:
            res = await session.request(method="GET", url=binance_uri)
            if res.status != 200:
                print(res.status, dict(res.headers))
                res.raise_for_status()
            timestamp = clock_ms()
            res_json = await res.text()
        JSON_object = json.loads(res_json)

        prices = [float(price) for price, quantity in JSON_object['bids']]
        weights = [float(quantity) for price, quantity in JSON_object['bids']]
        buy = np.average(prices, weights=weights)

        prices = [float(price) for price, quantity in JSON_object['asks']]
        weights = [float(quantity) for price, quantity in JSON_object['asks']]
        sell = np.average(prices, weights=weights)

        await q.put((symbol, buy, sell, timestamp))

In [33]:
def init_symbol_dict(symbol_list):
    symbol_dict = {s:[0,0,0] for s in symbol_list}
    df_prices = pd.DataFrame(symbol_dict, index=['buy', 'sell', 'timestamp']).T.reset_index().rename(columns={'index':'symbol'})

    # Standarize Prices
    df_prices['first_asset'] = np.NaN
    for i in range(df_prices['symbol'].apply(len).max()):
        mask = df_prices['symbol'].str[0:i].isin(assets)
        df_prices.loc[mask, 'first_asset'] = df_prices[mask]['symbol'].str[0:i]
    assert df_prices['first_asset'].isna().sum()==0, 'There are missing first_asset'

    df_prices = df_prices.drop(index=df_prices[df_prices['first_asset'].apply(len)==df_prices['symbol'].apply(len)].index)
    df_prices['second_asset'] = df_prices.apply(lambda x: x['symbol'][len(x['first_asset']):], axis=1)
    assert (~df_prices['second_asset'].isin(assets)).sum()==0, 'There are missing second_asset'
    assert (~(df_prices['first_asset']+df_prices['second_asset'] == df_prices['symbol'])).sum()==0, 'Symbol does not equal to asset1+asset2'

    return df_prices.set_index('symbol').T.to_dict('list')

def get_all_asyncioq_elems(q):
    queue_elements = []
    while True:
        try:
            queue_elements.append(q.get_nowait())
        except asyncio.QueueEmpty:
            break
    return queue_elements

def update_symbol_dict(d, elems):
    for t in elems:
        k = t[0]
        v = t[1:4]
        d[k][1:4] = v
    return d

In [43]:
initial_currency = 'BTC'
symbol_list = [s for s in symbols if initial_currency in s]
symbol_dict = init_symbol_dict(symbol_list)

In [44]:
symbol_dict

{'1INCHBTC': [0, 0, 0, '1INCH', 'BTC'],
 'AAVEBTC': [0, 0, 0, 'AAVE', 'BTC'],
 'ACMBTC': [0, 0, 0, 'ACM', 'BTC'],
 'ADABTC': [0, 0, 0, 'ADA', 'BTC'],
 'ADXBTC': [0, 0, 0, 'ADX', 'BTC'],
 'AERGOBTC': [0, 0, 0, 'AERGO', 'BTC'],
 'AGIBTC': [0, 0, 0, 'AGI', 'BTC'],
 'AIONBTC': [0, 0, 0, 'AION', 'BTC'],
 'AKROBTC': [0, 0, 0, 'AKRO', 'BTC'],
 'ALGOBTC': [0, 0, 0, 'ALGO', 'BTC'],
 'ALICEBTC': [0, 0, 0, 'ALICE', 'BTC'],
 'ALPHABTC': [0, 0, 0, 'ALPHA', 'BTC'],
 'AMBBTC': [0, 0, 0, 'AMB', 'BTC'],
 'ANKRBTC': [0, 0, 0, 'ANKR', 'BTC'],
 'ANTBTC': [0, 0, 0, 'ANT', 'BTC'],
 'APPCBTC': [0, 0, 0, 'APPC', 'BTC'],
 'ARDRBTC': [0, 0, 0, 'ARDR', 'BTC'],
 'ARKBTC': [0, 0, 0, 'ARK', 'BTC'],
 'ARPABTC': [0, 0, 0, 'ARPA', 'BTC'],
 'ASRBTC': [0, 0, 0, 'ASR', 'BTC'],
 'ASTBTC': [0, 0, 0, 'AST', 'BTC'],
 'ATMBTC': [0, 0, 0, 'ATM', 'BTC'],
 'ATOMBTC': [0, 0, 0, 'ATOM', 'BTC'],
 'AUCTIONBTC': [0, 0, 0, 'AUCTION', 'BTC'],
 'AUDIOBTC': [0, 0, 0, 'AUDIO', 'BTC'],
 'AUTOBTC': [0, 0, 0, 'AUTO', 'BTC'],
 'AVABTC': [0, 0

In [48]:
max_timelapse = 500
min_change_value = 0.0001 # to avoid float precission errors on eigvector and API computation
io_queue = asyncio.Queue()
future = asyncio.gather(*(update_symbol_prices(io_queue, s) for s in symbol_list))

while True:
    try:
        time_i = clock_ms()
        queue_elements = get_all_asyncioq_elems(io_queue)
        # update symbol dict
        for t in queue_elements:
            k = t[0]
            v = t[1:4]
            symbol_dict[k][0:3] = v
            
        # filter prices by most recent ones
        df_prices = pd.DataFrame().from_dict(symbol_dict, 'index', columns=['buy', 'sell', 'timestamp', 'first_asset', 'second_asset'])
        df_prices['timestamp_delta'] = clock_ms() - df_prices['timestamp']
        df_prices = df_prices[(df_prices[['buy', 'sell']] > min_change_value).sum(axis=1)!=0]
        df_prices = df_prices[df_prices['timestamp_delta'] < max_timelapse]

        if len(df_prices)>3:
            # Convert to Currency Matrix
            final_assets = sorted(list(set(df_prices['first_asset']).union(set(df_prices['second_asset']))))

            matrix_lite = pd.DataFrame(np.identity(len(final_assets)), index=final_assets, columns=final_assets)

            for index, row in df_prices.iterrows():
                matrix_lite.loc[row['first_asset'], row['second_asset']] = row['buy']
                matrix_lite.loc[row['second_asset'], row['first_asset']] = 1/row['sell']

            currency_matrix = matrix_lite.astype('float')
            currency_matrix = currency_matrix.replace(0,1)

            # Compute Arbitrage
            fees = 0.00075
            currencies = currency_matrix.columns.to_list()

            A = currency_matrix.to_numpy().copy().T
            eigvals, eigvects = np.linalg.eig(A)

            idxmax = np.argmax(eigvals)
            valmax = eigvals[idxmax]
            vecmax = eigvects[:,idxmax]

            valapi = compute_API(valmax, len(A))


            if valapi>0:
                print(f'Arbitrage oportunity detected. API={valapi:.4}')
                print('Matrix shape: ', currency_matrix.shape)
                calculate_arbitrage(A, vecmax, fees, currencies=currencies)
                print('-'*25, f'Elapsed Time: {clock_ms()-time_i:4.3}')
                await asyncio.sleep(5)
                continue
            else:
                print(f'Arbitrage oportunity no detected. API={valapi:.4}')
        print('-'*25, f'Elapsed Time: {clock_ms()-time_i:4.3}')
        await asyncio.sleep(0.5)
    except Exception as e:
        future.cancel()
        raise e

------------------------- Elapsed Time: 5.74
------------------------- Elapsed Time:  7.2
------------------------- Elapsed Time: 7.39
------------------------- Elapsed Time:  8.6
------------------------- Elapsed Time: 7.67
Arbitrage oportunity detected. API=33.28
Matrix shape:  (6, 6)
Arbitrage orders: DIRECT
UAH -> BTC -> UAH : AER = 0.450% : Fee = 0.150% : Return = 0.300%
BUY  BTC/UAH(5.988996879733287e-07) in UAH   
SELL UAH/BTC(1662245.3328580966) in BTC   
1/5.989e-07*1.662e+06 = 1.0045
------------------------- Elapsed Time: 12.8
Arbitrage oportunity detected. API=150.1
Matrix shape:  (9, 9)
Arbitrage orders: DIRECT
IDRT -> BTC -> IDRT : AER = 0.534% : Fee = 0.150% : Return = 0.384%
BUY  BTC/IDRT(1.1552589962592381e-09) in IDRT  
SELL IDRT/BTC(861008123.8836387) in BTC   
1/1.155e-09*8.61e+08 = 1.0053
------------------------- Elapsed Time: 15.5
------------------------- Elapsed Time:  6.3
------------------------- Elapsed Time: 7.67
------------------------- Elapsed Time: 6.41

CancelledError: 

429 {'Content-Type': 'application/json;charset=UTF-8', 'Content-Length': '171', 'Connection': 'keep-alive', 'Date': 'Sat, 03 Apr 2021 14:45:54 GMT', 'Server': 'nginx', 'x-mbx-uuid': '159af2db-8224-489a-a247-70022c0ff77f', 'x-mbx-used-weight': '1201', 'x-mbx-used-weight-1m': '1201', 'Retry-After': '0', 'Strict-Transport-Security': 'max-age=31536000; includeSubdomains', 'X-Frame-Options': 'SAMEORIGIN', 'X-Xss-Protection': '1; mode=block', 'X-Content-Type-Options': 'nosniff', 'Content-Security-Policy': "default-src 'self'", 'X-Content-Security-Policy': "default-src 'self'", 'X-WebKit-CSP': "default-src 'self'", 'Cache-Control': 'no-cache, no-store, must-revalidate', 'Pragma': 'no-cache', 'Expires': '0', 'X-Cache': 'Error from cloudfront', 'Via': '1.1 1f29f0a7fafc0aad0353030bb57e3bdb.cloudfront.net (CloudFront)', 'X-Amz-Cf-Pop': 'CDG3-C1', 'X-Amz-Cf-Id': 'OAR5FDb--d3CmYqK8smGCrFpmYDA3a2gvHMBqapFgD6EkFpPbUC2sA=='}
429 {'Content-Type': 'application/json;charset=UTF-8', 'Content-Length': '171

_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
concurrent.futures._base.CancelledError
